In [32]:
import capstone
import keystone
import shutil
import struct

In [33]:
def make_idx_dict(names):
    d = {} 
    for i, n in enumerate(names):
        d[n] = i

    return d

In [34]:
class Const:
    HEADER_SIZE = 64 

In [ ]:
class ElfHeader:
    unpacked_data = None

    fields = [
        "e_ident",
        "e_type",
        "e_machine",
        "e_version",
        "e_entry",
        "e_phoff",
        "e_shoff",
        "e_flags",
        "e_ehsize",
        "e_phentsize",
        "e_phnum",
        "e_shentsize",
        "e_shnum",
        "e_shstrndx",
    ]

    idx_dict = make_idx_dict(fields)

    format = (
        # e_ident (16 bytes), e_type (2 bytes), e_machine (2 bytes), e_version (4 bytes)
        '< 16s H H I' +
        # e_entry (8 bytes), e_phoff (8 bytes), e_shoff (8 bytes), e_flags (4 bytes)
        'Q Q Q I' +
        # e_ehsize (2 bytes), e_phentsize (2 bytes), e_phnum (2 bytes), e_shentsize (2 bytes)
        'H H H H' +
        # e_shnum (2 bytes), e_shstrndx (2 bytes)
        'H H'
    )

    def print():
        for i, f in enumerate(ElfHeader.fields):
            print(f'{f}: {ElfHeader.unpacked_data[i]}')
    
    def read_elf_header():
        if ElfFile.data[:4] != b'\x7fELF':
            raise ValueError("Not a valid ELF file.")
        
        ElfHeader.unpacked_data = list(struct.unpack(ElfHeader.format, ElfFile.data[:Const.HEADER_SIZE]))

    def overwrite_elf_header(file_path):
        amd_machine = 0x003e
        ElfHeader.unpacked_data[2] = amd_machine

        packed_data = struct.pack(ElfHeader.format, *ElfHeader.unpacked_data)

        with open(file_path, 'wb') as f:
            f.write(packed_data)
            f.write(ElfFile.data[Const.HEADER_SIZE:])

    def get(name):
        idx = ElfHeader.idx_dict[name]
        return ElfHeader.unpacked_data[idx]

In [ ]:
class Rela:
    size = 0x18

    format = '<QQq'  # r_offset, r_info, r_addend

    fields = ['r_offset', 'r_info', 'r_addend']

    idx_dict = make_idx_dict(fields)

    def __init__(self, offset):
        self.unpacked_data = struct.unpack(Rela.format, ElfFile.data[offset : offset + Rela.size])
        self.offset = offset

    def print(self):
        for i, f in enumerate(Rela.fields):
            print(f'{f}: {self.unpacked_data[i]}')

    def overwrite_rela(self, offset):
        pass

    def get(self, name):
        idx = Rela.idx_dict[name]
        return self.unpacked_data[idx]

In [36]:
class SectionHeader:
    fields = [
        "sh_name",      # Section name (index into section header string table)
        "sh_type",      # Section type
        "sh_flags",     # Section attributes
        "sh_addr",      # Virtual address in memory
        "sh_offset",    # Offset in file
        "sh_size",      # Size of section
        "sh_link",      # Link to other section
        "sh_info",      # Miscellaneous information
        "sh_addralign", # Address alignment boundary
        "sh_entsize"    # Size of entries, if section has table
    ]

    format = (
        # sh_name (4 bytes), sh_type (4 bytes), sh_flags (8 bytes), sh_addr (8 bytes)
        '< I I Q Q' +  
        # sh_offset (8 bytes), sh_size (8 bytes), sh_link (4 bytes), sh_info (4 bytes)
        'Q Q I I' +    
        # sh_addralign (8 bytes), sh_entsize (8 bytes)
        'Q Q' 
    )

    idx_dict = make_idx_dict(fields)

    shstroff = None
    
    def __init__(self, offset):
        self.unpacked_data = list(
            struct.unpack(
                SectionHeader.format, 
                ElfFile.data[offset : offset + ElfHeader.get('e_shentsize')],
            )
        )

        sh_off = self.get('sh_offset')
        self.section_data = ElfFile.data[sh_off : sh_off + self.get('sh_size')]

        self.name = None

    def print(self):
        for i, f in enumerate(SectionHeader.fields):
            print(f'{f}: {self.data[i]}')

    def get(self, name):
        idx = SectionHeader.idx_dict[name]
        return self.unpacked_data[idx]
    
    def set_name(self, verbose = True):
        name_offset = self.get('sh_name') + SectionHeader.shstroff 
        name_end = ElfFile.data.find(b'\x00', name_offset)
        name_len = name_end - name_offset

        self.name = struct.unpack(f'{name_len}s', ElfFile.data[name_offset : name_end])

        if verbose:
            print(self.name)

In [37]:
class Translator:
    def disassemble_code(code_section):
        # AArch64 architecture
        md = capstone.Cs(capstone.CS_ARCH_ARM64, capstone.CS_MODE_ARM)
        md._detail = True

        instructions = md.disasm(code_section, 0)

        for insn in instructions:
            print(f"0x{insn.address:x}:\t{insn.mnemonic}\t{insn.op_str}")

        # print('')

        # for insn in instructions:
            print(f"Instruction: {insn.mnemonic} {insn.op_str}")
            
            # Checking for immediate value offset
            try:
                for i, operand in enumerate(insn.operands):
                    if operand.type == capstone.ARM64_OP_IMM:
                        print(f"Immediate offset: {insn.imm_offset}")
                        print(f"Immediate value: {operand.value.imm}")

                    if operand.type == capstone.ARM64_OP_MEM:
                        # Check for displacement
                        print(f"Displacement offset: {insn.disp_offset}")
                        print(f"Displacement value: {operand.value.mem.disp}")
            except:
                print()

    def assemble_code(code):
        # separate assembly instructions by ; or \n
        CODE = b"INC ecx; DEC edx"
        
        try:
            ks = keystone.Ks(keystone.KS_ARCH_X86, keystone.KS_MODE_64)
            encoding, count = ks.asm(CODE)
            print("%s = %s (number of statements: %u)" %(CODE, encoding, count))
        except keystone.KsError as e:
            print("ERROR: %s" %e)

In [38]:
class ElfFile:
    data = None
    section_headers = []

    def setup(file_path):
        with open(file_path, 'rb') as f:
            ElfFile.data = f.read()

    def read_elf_header():
        ElfHeader.read_elf_header()

    def read_section_headers():
        for i in range(ElfHeader.get('e_shnum')):
            offset = ElfHeader.get('e_shoff') + i * ElfHeader.get('e_shentsize')

            ElfFile.section_headers += [SectionHeader(offset)]
        
        shstrns = ElfFile.section_headers[ElfHeader.get('e_shstrndx')]
        SectionHeader.shstroff = shstrns.get('sh_offset')

        for sh in ElfFile.section_headers:
            sh.set_name()

    def find_code_sections():
        for sh in ElfFile.section_headers: 
            if sh.get('sh_type')== 1:  # SHT_PROGBITS 
                Translator.disassemble_code(sh.section_data)

In [39]:
input = 'test-aarch64.o'  
output = 'out.o'
good_output = 'test-aarch64-x64.o'  

shutil.copy(input, output)

ElfFile.setup(input)
ElfFile.read_elf_header()
ElfFile.read_section_headers()
ElfFile.find_code_sections()

# disassemble_code(section)
# assemble_code(None)

(b'',)
(b'.text',)
(b'.rela.text',)
(b'.data',)
(b'.bss',)
(b'.rodata',)
(b'.comment',)
(b'.note.GNU-stack',)
(b'.eh_frame',)
(b'.rela.eh_frame',)
(b'.symtab',)
(b'.strtab',)
(b'.shstrtab',)


ValueError: NULL pointer access